# Setup & imports

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Optional modeling libs

In [2]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

## Notebook plotting settings

In [3]:
%matplotlib inline
plt.style.use('default')
sns.set_context('talk')

# Helper Function

In [10]:
def try_load_data(data_dir='data'):
    """
    Try to find a dataset file in data/ and load it.
    Accepts: online_retail_II.xlsx, online_retail.csv, OnlineRetail.csv, etc.
    Returns pandas DataFrame.
    """
    with open(rf'{data_dir}\online_retail_II.csv') as f:
        return pd.read_csv(f)

In [31]:
def save_fig(fig, fname):
    """Save figure into images/ folder (create if missing)."""
    os.makedirs('images', exist_ok=True)
    path = os.path.join('images', fname)
    fig.savefig(path, bbox_inches='tight', dpi=150)
    print("Saved:", path)

# Loading Data

In [11]:
df = try_load_data()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,01-12-2009 07:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,01-12-2009 07:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,01-12-2009 07:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,01-12-2009 07:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,01-12-2009 07:45,1.25,13085.0,United Kingdom
